In [37]:
# !python /data/benny_cai/miniconda3/envs/brax/lib/python3.9/site-packages/brax/v1/tools/urdf_converter.py --xml_model_path 'legged_studio_a1_more_colli.urdf' --config_path 'a1'

I0125 20:41:11.544692 140288389826368 urdf_converter.py:58] Loading urdf model from legged_studio_a1_more_colli.urdf


In [1]:
# !python /data/benny_cai/miniconda3/envs/brax/lib/python3.9/site-packages/brax/v1/tools/mujoco_converter.py --xml_model_path 'a1_mjcf.txt' --config_path 'a1_mjcf'

## With interpolation

In [1]:
from brax.positional import pipeline
from brax.io import mjcf

from IPython.display import HTML, clear_output
import jax
from jax import numpy as jp
from brax.io import html
import json
import numpy as np
import copy

In [2]:
def standardize_quaternion(quat):
    mask = quat[..., -1] < 0
    quat[mask] = -quat[mask]
    return quat

def quat_slerp(quat0, quat1, fraction, spin=0, shortestpath=True, eps=1e-6):
    if isinstance(fraction, (int, float)):
        fraction = np.array([fraction])
    batch_size = quat0.shape[0]
    q = np.zeros_like(quat0)
    q0 = quat0[..., :4] / \
        np.linalg.norm(quat0[..., :4], axis=-1).reshape([batch_size, 1])
    q1 = quat1[..., :4] / \
        np.linalg.norm(quat1[..., :4], axis=-1).reshape([batch_size, 1])
    fraction_zero = np.squeeze(fraction == 0.0)
    fraction_one = np.squeeze(fraction == 1.0)
    mask = np.logical_and(
        np.logical_not(fraction_zero),
        np.logical_not(fraction_one)
    )
    q[fraction_zero] = q0[fraction_zero]
    q[fraction_one] = q1[fraction_one]
    d = np.squeeze(np.matmul(q0.reshape([-1, 1, q0.shape[1]]),
                                q1.reshape([-1, q1.shape[1], 1])))
    d[np.logical_not(mask)] = 0.0  # set to dummy value
    q[np.logical_and(mask, abs(abs(d) - 1.0) < eps)
    ] = q0[np.logical_and(mask, abs(abs(d) - 1.0) < eps)]
    mask = np.logical_and(
        mask,
        abs(abs(d) - 1.0) >= eps
    )
    d = d[mask]
    if shortestpath:
        # invert rotation
        d[d < 0.0] = -d[d < 0.0]
        q1[mask][d < 0.0] *= -1.0
    d = d.reshape([-1, 1])
    angle = np.arccos(d) + spin * np.pi
    isin = 1.0 / np.sin(angle)
    q0[mask] *= np.sin((1.0 - fraction[mask]) * angle) * isin
    q1[mask] *= np.sin(fraction[mask] * angle) * isin
    q0[mask] += q1[mask]
    q[mask] = q0[mask]
    return q

In [3]:
with open('a1_mjcf.txt', 'r') as file:
    config = file.read()

m = mjcf.loads(config, asset_path='')

In [4]:
file = 'a1_origin_motions/trot.txt'
with open(file, 'r') as f:
    data = json.load(f)
frames = data['Frames']
frames = np.array(frames)
frames.shape

(33, 19)

In [5]:
interpolated_frames = []
for i in range(frames.shape[0]-1):
    frame0, frame1 = frames[i], frames[i+1]
    root_pos0 = frame0[0:3]
    root_pos1 = frame1[0:3]
    root_rot0 = frame0[3:7]
    root_rot1 = frame1[3:7]
    joints0 = frame0[7:19]
    joints1 = frame1[7:19]
    _root_rot0 = root_rot0[np.newaxis, ...]
    _root_rot1 = root_rot1[np.newaxis, ...]
    interpolated_frames.append(frame0)
    for j in range(1,10):
        blend = j / 10
        blend_root_pos = (1.0 - blend) * root_pos0 + blend * root_pos1
        blend_joints = (1.0 - blend) * joints0 + blend * joints1
        blend_root_rot = quat_slerp(_root_rot0, _root_rot1, blend)
        blend_root_rot /= np.linalg.norm(blend_root_rot, axis=-1).reshape([blend_root_rot.shape[0], 1])
        blend_root_rot = standardize_quaternion(blend_root_rot)[0]
        blend_root_rot = np.round(blend_root_rot, decimals=5)
        interpolated_frames.append(np.hstack((blend_root_pos, blend_root_rot, blend_joints)))
interpolated_frames.append(frame1)

In [6]:
interpolated_frames = np.array(interpolated_frames)
interpolated_frames.shape

(321, 19)

In [7]:
jit_env_reset = jax.jit(pipeline.init)
jit_env_step = jax.jit(pipeline.step)

rng = jax.random.PRNGKey(seed=1)
rollout = []
for frame in interpolated_frames:
    frame[3], frame[6] = frame[6], frame[3]
    state = jit_env_reset(m, frame, jp.zeros(m.qd_size()))
    rollout.append(state)

In [9]:
# HTML(html.render(m, rollout))

In [10]:
# serialize data, positional:
data = []
for roll in rollout:
    QP = np.hstack((
        roll.q, 
        roll.qd, 
        roll.x.pos.flatten(), 
        roll.x.rot.flatten(), 
        roll.xd.vel.flatten(), 
        roll.xd.ang.flatten(),
        roll.x_i.pos.flatten(), 
        roll.x_i.rot.flatten(), 
        roll.xd_i.vel.flatten(), 
        roll.xd_i.ang.flatten(),
        roll.j.pos.flatten(), 
        roll.j.rot.flatten(), 
        roll.jd.vel.flatten(), 
        roll.jd.ang.flatten(),
        roll.a_p.pos.flatten(), 
        roll.a_p.rot.flatten(), 
        roll.a_c.pos.flatten(), 
        roll.a_c.rot.flatten(),
        roll.mass
    ))
    data.append(QP)

In [11]:
data = np.array(data)
data.shape

(321, 739)

In [12]:
np.save('a1_ref_motion/trot.npy', data)

## No interpolation

In [1]:
from brax.positional import pipeline
from brax.io import mjcf

from IPython.display import HTML, clear_output
import jax
from jax import numpy as jp
from brax.io import html
import json
import numpy as np

In [2]:
with open('a1_mjcf.txt', 'r') as file:
    config = file.read()

m = mjcf.loads(config, asset_path='')

In [9]:
file = 'a1_origin_motions/trot.txt'
with open(file, 'r') as f:
    data = json.load(f)
frames = data['Frames']
frames = np.array(frames)
frames.shape

(33, 19)

In [10]:
jit_env_reset = jax.jit(pipeline.init)
jit_env_step = jax.jit(pipeline.step)

rng = jax.random.PRNGKey(seed=1)
# state = jit_env_reset(m, m.init_q, jp.zeros(m.qd_size()))
rollout = []
for frame in frames:
    frame[3], frame[4], frame[5], frame[6] = frame[6], frame[3], frame[4], frame[5]
    # frame[3], frame[6] = frame[6], frame[3]
    state = jit_env_reset(m, frame, jp.zeros(m.qd_size()))
    rollout.append(state)

In [11]:
HTML(html.render(m.replace(dt=0.02), rollout))

In [33]:
# # serialize data, generalized:
# data = []
# for roll in rollout:
#     QP = np.hstack((roll.q, roll.qd, roll.x.pos.flatten(), 
#                     roll.x.rot.flatten(), roll.xd.vel.flatten(), roll.xd.ang.flatten(),
#                     roll.root_com.flatten(),
#                     roll.cinr.transform.pos.flatten(),
#                     roll.cinr.transform.rot.flatten(),
#                     roll.cinr.i.flatten(),
#                     roll.cinr.mass,
#                     roll.cd.vel.flatten(),
#                     roll.cd.ang.flatten(),
#                     roll.cdof.vel.flatten(),
#                     roll.cdof.ang.flatten(),
#                     roll.cdofd.vel.flatten(),
#                     roll.cdofd.ang.flatten(),
#                     roll.mass_mx.flatten(),
#                     roll.mass_mx_inv.flatten(),
#                     roll.con_jac.flatten(),
#                     roll.con_diag,
#                     roll.con_aref,
#                     roll.qf_smooth,
#                     roll.qf_constraint,
#                     roll.qdd
#                    ))
#     data.append(QP)

In [10]:
# serialize data, positional:
data = []
for roll in rollout:
    QP = np.hstack((
        roll.q, 
        roll.qd, 
        roll.x.pos.flatten(), 
        roll.x.rot.flatten(), 
        roll.xd.vel.flatten(), 
        roll.xd.ang.flatten(),
        roll.x_i.pos.flatten(), 
        roll.x_i.rot.flatten(), 
        roll.xd_i.vel.flatten(), 
        roll.xd_i.ang.flatten(),
        roll.j.pos.flatten(), 
        roll.j.rot.flatten(), 
        roll.jd.vel.flatten(), 
        roll.jd.ang.flatten(),
        roll.a_p.pos.flatten(), 
        roll.a_p.rot.flatten(), 
        roll.a_c.pos.flatten(), 
        roll.a_c.rot.flatten(),
        roll.mass
    ))
    data.append(QP)

In [11]:
data = np.array(data)
data.shape

(33, 739)

In [12]:
np.save('a1_ref_motion/trot.npy', data)

In [31]:
dir(rollout[0])

['T',
 '__add__',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_flax_dataclass',
 'a_c',
 'a_p',
 'concatenate',
 'contact',
 'index_set',
 'index_sum',
 'j',
 'jd',
 'mass',
 'q',
 'qd',
 'replace',
 'reshape',
 'select',
 'slice',
 'take',
 'tree_replace',
 'vmap',
 'x',
 'x_i',
 'xd',
 'xd_i']

In [39]:
rollout[10].contact

In [34]:
rollout[0].x_i.pos - rollout[0].x.pos

Array([[-8.68828793e-05,  4.12178272e-03, -2.51710415e-04],
       [-3.29609215e-03, -7.08241016e-04, -3.27527523e-05],
       [-1.00523382e-02,  2.26075128e-02, -2.53679752e-02],
       [ 1.30997702e-01,  2.60286033e-03,  1.65096223e-02],
       [-3.32412124e-03,  5.62351197e-04, -3.05473804e-05],
       [-2.12564766e-02, -2.30624303e-02, -1.64915025e-02],
       [ 1.10549971e-01,  1.24979019e-03, -7.22280219e-02],
       [ 3.32412124e-03, -5.62336296e-04,  3.01599503e-05],
       [-2.28128880e-02,  2.30522826e-02, -1.42775476e-02],
       [ 9.70614851e-02,  9.59122926e-03, -8.90327096e-02],
       [ 3.29473615e-03,  7.04083592e-04,  1.25974417e-04],
       [-2.35372782e-02, -2.19015554e-02, -1.48997307e-02],
       [ 1.04690775e-01,  7.94543326e-03, -8.01027641e-02]],      dtype=float32)

In [2]:
rollouts = np.load('/data/benny_cai/diffmimic/logs/exp_300_32_0.0003_10_0.4_0_threshold_0.4_True_54_60_600_True_0.02_0.5_0.01_0.01_trot/eval_traj_10.npy')

In [3]:
rollouts.shape

(32, 32, 739)

In [5]:
# rollouts[0]

### Try

In [1]:
from brax.positional import pipeline
from brax.io import mjcf

from IPython.display import HTML, clear_output
import jax
from jax import numpy as jp
from brax.io import html

In [2]:
with open('a1_mjcf.txt', 'r') as file:
    config = file.read()

m = mjcf.loads(config, asset_path='')
# m = m.replace(dt=0.02) # default is 0.002

In [4]:
jit_env_reset = jax.jit(pipeline.init)
jit_env_step = jax.jit(pipeline.step)

In [5]:
from brax.envs import base
class A1Mimic(base.PipelineEnv):
    def __init__(self):
        self.sys = mjcf.loads(config, asset_path='')
        # self.sys = sys.replace(dt=0.02)
        super().__init__(sys=self.sys, backend='positional', n_frames=10)

    def reset(self):
        return super().pipeline_init(self.sys.init_q, jp.zeros(self.sys.qd_size()))

    def step(self, state, action: jp.ndarray):
        return super().pipeline_step(state, action)

In [6]:
a1 = A1Mimic()

In [9]:
a1.dt

Array(0.02, dtype=float32, weak_type=True)

In [11]:
rollout = []
rng = jax.random.PRNGKey(seed=1)
state = a1.reset()
for i in range(2):
    act = jp.array([0, 0.5, -1.5, 0, 0.5, -1.5, 0, 0.5, -1.2, 0, 0.5, -1.2]) # target angle
    state = a1.step(state, act)
    rollout.append(state)
rollout.append(state)

In [5]:
# HTML(html.render(a1.sys.replace(dt=0.02), rollout))

In [33]:
rollout = []
rng = jax.random.PRNGKey(seed=1)
state = a1.reset()
for i in range(1):
    print(i)
    # print(jp.round(state.q[7:], decimals=1))
    rollout.append(state)
    # act = 10 * jp.sin(i / 100) * jp.ones(m.act_size())
    # act = jp.array([0, 0.9, -1.8, 0, 0.9, -1.8, 0, 0.9, -1.8, 0, 0.9, -1.8])
    act = jp.array([0, 0.5, -1.5, 0, 0.5, -1.5, 0, 0.5, -1.2, 0, 0.5, -1.2]) # target angle
    state = a1.step(state, act)
rollout.append(state)

0


In [38]:
# HTML(html.render(m, rollout))

In [33]:
dir(m)

['T',
 '__add__',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_flax_dataclass',
 '_model',
 'act_size',
 'actuator',
 'ang_damping',
 'baumgarte_erp',
 'collide_scale',
 'concatenate',
 'density',
 'dof',
 'dof_link',
 'dof_ranges',
 'dt',
 'enable_fluid',
 'geom_masks',
 'geoms',
 'get_mjx_model',
 'get_model',
 'gravity',
 'index_set',
 'index_sum',
 'init_q',
 'joint_scale_ang',
 'joint_scale_pos',
 'link',
 'link_names',
 'link_parents',
 'link_types',
 'matrix_inv_iterations',
 'num_links',
 'q_idx',
 'q_size',
 'qd_idx',
 'qd_size',
 'rep

In [34]:
m.link_names

['trunk',
 'FR_hip',
 'FR_thigh',
 'FR_calf',
 'FL_hip',
 'FL_thigh',
 'FL_calf',
 'RR_hip',
 'RR_thigh',
 'RR_calf',
 'RL_hip',
 'RL_thigh',
 'RL_calf']

In [31]:
dir(m.actuator)

['T',
 '__add__',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_flax_dataclass',
 'bias_q',
 'bias_qd',
 'concatenate',
 'ctrl_range',
 'force_range',
 'gain',
 'gear',
 'index_set',
 'index_sum',
 'q_id',
 'qd_id',
 'replace',
 'reshape',
 'select',
 'slice',
 'take',
 'tree_replace',
 'vmap']

In [29]:
m.link.index_set

<bound method Base.index_set of Link(transform=Transform(pos=Array([[ 0.     ,  0.     ,  0.     ],
       [ 0.183  , -0.047  ,  0.     ],
       [ 0.     , -0.08505,  0.     ],
       [ 0.     ,  0.     , -0.2    ],
       [ 0.183  ,  0.047  ,  0.     ],
       [ 0.     ,  0.08505,  0.     ],
       [ 0.     ,  0.     , -0.2    ],
       [-0.183  , -0.047  ,  0.     ],
       [ 0.     , -0.08505,  0.     ],
       [ 0.     ,  0.     , -0.2    ],
       [-0.183  ,  0.047  ,  0.     ],
       [ 0.     ,  0.08505,  0.     ],
       [ 0.     ,  0.     , -0.2    ]], dtype=float32), rot=Array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)), joint=Transform(pos=Array([[0., 0., 0.],
       [0., 0., 0.],
      

In [61]:
m.link.constraint_stiffness

Array([2000., 2000., 2000., 2000., 2000., 2000., 2000., 2000., 2000.,
       2000., 2000., 2000., 2000.], dtype=float32)

In [60]:
m.link.constraint_limit_stiffness

Array([1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
       1000., 1000., 1000., 1000.], dtype=float32)

In [31]:
m.actuator.gain

Array([100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
       100.], dtype=float32)

In [32]:
m.gravity

Array([ 0.  ,  0.  , -9.81], dtype=float32)

In [39]:
m.ang_damping

Array(0., dtype=float32)

In [40]:
m.baumgarte_erp

Array(0.1, dtype=float32)

In [42]:
m.vel_damping

Array(0., dtype=float32)

In [41]:
m.joint_scale_ang

Array(0.2, dtype=float32)

In [63]:
m.link.inertia

Inertia(transform=Transform(pos=Array([[ 0.00000e+00,  4.10000e-03, -5.00000e-04],
       [-3.31100e-03, -6.35000e-04,  3.10000e-05],
       [-3.23700e-03,  2.23270e-02, -2.73260e-02],
       [ 4.72659e-03,  0.00000e+00, -1.31975e-01],
       [-3.31100e-03,  6.35000e-04,  3.10000e-05],
       [-3.23700e-03, -2.23270e-02, -2.73260e-02],
       [ 4.72659e-03,  0.00000e+00, -1.31975e-01],
       [ 3.31100e-03, -6.35000e-04,  3.10000e-05],
       [-3.23700e-03,  2.23270e-02, -2.73260e-02],
       [ 4.72659e-03,  0.00000e+00, -1.31975e-01],
       [ 3.31100e-03,  6.35000e-04,  3.10000e-05],
       [-3.23700e-03, -2.23270e-02, -2.73260e-02],
       [ 4.72659e-03,  0.00000e+00, -1.31975e-01]], dtype=float32), rot=Array([[-1.8256460e-03,  7.0638072e-01, -6.4208568e-04,  7.0782948e-01],
       [ 5.0752789e-01,  5.0626791e-01,  4.9150690e-01,  4.9449891e-01],
       [ 9.9912524e-01, -2.5639306e-03, -4.0953111e-02, -8.0609117e-03],
       [ 7.0688641e-01,  1.7653009e-02,  1.7653009e-02,  7.068864

In [45]:
dir(m.get_model())

['__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_address',
 'actuator',
 'actuator_acc0',
 'actuator_actadr',
 'actuator_actearly',
 'actuator_actlimited',
 'actuator_actnum',
 'actuator_actrange',
 'actuator_biasprm',
 'actuator_biastype',
 'actuator_cranklength',
 'actuator_ctrllimited',
 'actuator_ctrlrange',
 'actuator_dynprm',
 'actuator_dyntype',
 'actuator_forcelimited',
 'actuator_forcerange',
 'actuator_gainprm',
 'actuator_gaintype',
 'actuator_gear',
 'actuator_group',
 'actuator_length0',
 'actuator_lengthrange',
 'actuator_plugin',
 'actuator_trnid',
 'actuator_trntype',
 'actuator_user',
 'body',
 'body_bvhadr',
 'body_

In [48]:
m.get_model().actuator_gaintype

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [40]:
from IPython.display import HTML, clear_output
import jax
from jax import numpy as jp

import brax
from brax.v1.envs import env
from brax.v1.io import json
from brax.v1.io import html

In [41]:
with open('a1', 'r') as file:
    content = file.read()

In [42]:
class A1(env.Env):
    def __init__(self, system_config):
        super().__init__(config=system_config)

    def reset(self, rng: jp.ndarray) -> env.State:
        qp = self.sys.default_qp()
        pos = qp.pos
        info = self.sys.info(qp)
        obs = jp.zeros(1)
        reward, done, zero = jp.zeros(3)
        metrics = {
            'torsoIsUp': zero,
            'torsoHeight': zero
        }
        info = {'rng': rng}
        return env.State(qp, obs, reward, done, metrics, info)

    def step(self, state: env.State, action: jp.ndarray) -> env.State:
        qp, info = self.sys.step(state.qp, action)
        return state.replace(qp=qp)

In [43]:
a1 = A1(content)

/data/benny_cai/miniconda3/envs/brax/lib/python3.9/site-packages/brax/v1/physics/base.py:193: UserWarning: dynamics_mode not specified, but joint.stiffness >0. Setting dynamics_mode="legacy_spring".
  warnings.warn('dynamics_mode not specified, but joint.stiffness >0. '


In [14]:
# env_state = a1.reset(rng=jax.random.PRNGKey(0))
# HTML(html.render(a1.sys, [a1.sys.default_qp()]))

In [58]:
env_state = a1.reset(rng=jax.random.PRNGKey(0))

# rollout = []
# for i in range(2):  # You can adjust the number of simulation steps
#     print(i)
#     # # Take a random action
#     # action = jax.random.uniform(jax.random.PRNGKey(0), shape=(a1.action_size,))
#     # action = jp.insert(action, 0,0)
#     # action = jp.append(action, 0)
#     action = jp.ones(a1.action_size,) * (-0.2)
    
#     # Step the environment
#     env_state = a1.step(env_state, action)

#     rollout.append(env_state.qp)

# rollout = []
# action = jp.ones(a1.action_size,) * (-0.01) * 0
# env_state = a1.step(env_state, action)
# rollout.append(env_state.qp)

# action = jp.ones(a1.action_size,) * (-0.01)
# env_state = a1.step(env_state, action)
# rollout.append(env_state.qp)

In [6]:
# HTML(html.render(a1.sys, rollout))

In [7]:
a1.action_size

12

In [10]:
a1.sys.num_joints

12

In [8]:
a1.sys.body.index

{'trunk': 0,
 'FR_hip': 1,
 'FR_upper': 2,
 'FR_lower': 3,
 'FL_hip': 4,
 'FL_upper': 5,
 'FL_lower': 6,
 'RR_hip': 7,
 'RR_upper': 8,
 'RR_lower': 9,
 'RL_hip': 10,
 'RL_upper': 11,
 'RL_lower': 12,
 'floor': 13}

In [11]:
a1.sys.body.inertia

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [9]:
a1.sys.default_qp()

QP(pos=array([[ 0.        ,  0.        ,  0.41992316],
       [ 0.183     , -0.047     ,  0.41992316],
       [ 0.183     , -0.13205   ,  0.41992316],
       [ 0.17751757, -0.13205   ,  0.21999831],
       [ 0.183     ,  0.047     ,  0.41992316],
       [ 0.183     ,  0.13205   ,  0.41992316],
       [ 0.17751757,  0.13205   ,  0.21999831],
       [-0.183     , -0.047     ,  0.41992316],
       [-0.183     , -0.13205   ,  0.41992316],
       [-0.18848243, -0.13205   ,  0.21999831],
       [-0.183     ,  0.047     ,  0.41992316],
       [-0.183     ,  0.13205   ,  0.41992316],
       [-0.18848243,  0.13205   ,  0.21999831],
       [ 0.        ,  0.        ,  0.        ]]), rot=array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 9.99906050e-01,  3.04364422e-18,  1.37073550e-02,
         0.00000000e+00],
       [ 9.99997886e-01, -4.56560494e-19, -2.05616567e-03,
 